# PETS for FINRL

In [1]:
%load_ext autoreload
%autoreload 2

### Install libraries

In [2]:
# ## install finrl library
!pip install wrds
!pip install swig
!pip install gymnasium
!pip install stable-baselines3
!pip install stockstats
!pip install alpaca-trade-api
!pip install exchange-calendars
!pip install pyfolio
!pip install dotmap
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


In [3]:
import pandas as pd
from stable_baselines3.common.logger import configure

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.main import check_and_make_directories
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

check_and_make_directories([TRAINED_MODEL_DIR])

2024-11-27 14:31:52.417560: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 14:31:52.423670: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-27 14:31:52.435837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 14:31:52.457076: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 14:31:52.463471: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attemptin

In [4]:
train = pd.read_csv('train_data.csv')

# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following two lines.
train = train.set_index(train.columns[0])
train.index.names = ['']

In [5]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [6]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [8]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [ ]:
#%%bash
#cd handful_of_trials_pytorch
#pwd
import sys
sys.path.insert(0, "/home/hice1/skamath36/scratch/drl/FinRL-test/examples/handful_of_trials_pytorch")
print(sys.path)

In [51]:
from mbexp import main

main("finrl","MPC", [], [], "./log", env_train)


{'ctrl_cfg': {'env': <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x1553edbcfa60>,
              'opt_cfg': {'ac_cost_fn': <function FinRLConfigModule.ac_cost_fn at 0x1553a06ce950>,
                          'cfg': {'alpha': 0.1,
                                  'max_iters': 5,
                                  'num_elites': 40,
                                  'popsize': 400},
                          'mode': 'CEM',
                          'obs_cost_fn': <function FinRLConfigModule.obs_cost_fn at 0x1553a06ce290>,
                          'plan_hor': 25},
              'prop_cfg': {'mode': 'TSinf',
                           'model_init_cfg': {'model_constructor': <bound method FinRLConfigModule.nn_constructor of <finrl.FinRLConfigModule object at 0x1553a06fffa0>>,
                                              'num_nets': 5},
                           'model_train_cfg': {'epochs': 5},
                           'npart': 20,
                           'ob

AttributeError: module 'torch._dynamo.variables' has no attribute 'LazyVariableTracker'